In [1]:
from dataclasses import dataclass
from typing import Callable
from lmfit import  minimize
import pandas as pd
import os, glob
from pathlib import Path

from fit_utils import baseline_data, fit_data, plot_data, extract_data

BaselineFunction = Callable[[pd.DataFrame, pd.DataFrame, \
                    pd.DataFrame, pd.DataFrame], pd.DataFrame]
FitFunction = Callable[[pd.DataFrame, pd.DataFrame], minimize]
PlotFunction = Callable[[pd.DataFrame, pd.DataFrame, minimize, minimize], None]
ExtractFunction = Callable[[minimize, minimize, pd.DataFrame], float]

BACKGROUND_PATH = Path(r"C:\Users\hwahab\Desktop\TEST2__\data")
CAMPAIGN_PATH = Path(r"C:\Users\hwahab\Desktop\TEST2__\Campaigns\2022-04-14-Series1_")

In [2]:
oneD = [i for i in CAMPAIGN_PATH.glob("**/*1D.csv")]
twoD = [i for i in CAMPAIGN_PATH.glob("**/*2D.csv")]

In [3]:
@dataclass
class RatioBot:

    baseline_process: BaselineFunction
    fit_process: FitFunction
    plot_process: PlotFunction
    extract_process: ExtractFunction

    def run(self) -> float:
        d1_ = pd.read_csv(BACKGROUND_PATH / "background1D.csv")
        d2_ = pd.read_csv(BACKGROUND_PATH / "background2D.csv")
        for one, two in zip(oneD, twoD):
            print('LOADING DATA')
            d1 = pd.read_csv(one)
            d2 = pd.read_csv(two)

            d1, d2 = self.baseline_process(d1, d2, d1_, d2_)
            out1, out2 = self.fit_process(d1, d2)
            ratio = self.extract_process(out1, out2, d1)
            print(f'GD RATIO: {ratio}')
            self.plot_process(d1, d2, out1, out2, one) 
        return ratio    

        # print('LOADING DATA')
        # d1 = pd.read_csv("col1 line "+str(line) + " Point " + str(spot) + \
        #     " iteration "+str(idx) + " foreground1D.csv")
        # d2 = pd.read_csv("col1 line "+str(line) + " Point " + str(spot) + \
        #     " iteration "+str(idx) + " foreground2D.csv")
        # d1_ = pd.read_csv("../../data/background1D.csv")
        # d2_ = pd.read_csv("../../data/background2D.csv")
        
        # d1, d2 = self.baseline_process(d1, d2, d1_, d2_)
        # out1, out2 = self.fit_process(d1, d2)
        # ratio = self.extract_process(out1, out2, d1)
        # print(f'GD RATIO: {ratio}')
        # return ratio


In [4]:
bot = RatioBot(baseline_data, fit_data, plot_data, extract_data)
bot.run(1, 2,)